hypothesis test for the  rural I-40 Madison and Henderson crashes.csv

Import packages

In [ ]:
from itertools import combinations
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd



The bootstrapping function for test mltigroups

In [ ]:
def run_multigroup_bootstrap_test(df, feature, levels, geo_features, catboost_cat_features=None, seed=42):
    def bootstrap_test(preds1, preds2, n=10000):
        np.random.seed(seed)
        s1 = np.random.choice(preds1, (n, len(preds1)), replace=True).mean(axis=1)
        s2 = np.random.choice(preds2, (n, len(preds2)), replace=True).mean(axis=1)
        diff = s1 - s2
        p_val = np.mean(diff <= 0)
        return {
            "mean_1": preds1.mean(),
            "mean_2": preds2.mean(),
            "ci_1": np.percentile(s1, [2.5, 97.5]),
            "ci_2": np.percentile(s2, [2.5, 97.5]),
            "p_value": p_val
        }

    df = df.copy()

    # Ensure geo_features are numeric for filtering and modeling
    for col in geo_features:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Create Injurious outcome
    df = df[df[feature].isin(levels) & df["Crash Type"].notnull()]
    if df.empty:
        print(f"[{feature}] has no valid rows after filtering with levels: {levels}")
        return None

    df["Injurious"] = df["Crash Type"].str.lower().apply(
        lambda x: 1 if "injury" in x or "fatal" in x or "serious injury" in x else 0
    )

    # Train models once
    X = df[geo_features]
    y = df["Injurious"]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)
    ros = RandomOverSampler(random_state=seed)
    X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

    # Logistic model (can handle numeric or encoded categoricals)
    preprocessor = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), geo_features)])
    log_model = Pipeline([('preprocessor', preprocessor), ('classifier', LogisticRegression(max_iter=1000))])
    log_model.fit(X_train_res, y_train_res)

    # CatBoost: convert features to string before training
    X_train_res_str = X_train_res.astype(str)
    cat_model = CatBoostClassifier(verbose=0, random_state=seed, cat_features=catboost_cat_features or geo_features)
    cat_model.fit(X_train_res_str, y_train_res)

    # Loop through all level pairs
    results = []
    for lvl1, lvl2 in combinations(levels, 2):
        g1 = df[df[feature] == lvl1]
        g2 = df[df[feature] == lvl2]
        if len(g1) < 10 or len(g2) < 10:
            continue

        X1 = g1[geo_features]
        X2 = g2[geo_features]

        # Logistic predictions
        log1 = log_model.predict(X1)
        log2 = log_model.predict(X2)

        # CatBoost predictions (convert to str)
        cat1 = cat_model.predict(Pool(X1.astype(str), cat_features=geo_features))
        cat2 = cat_model.predict(Pool(X2.astype(str), cat_features=geo_features))

        # Bootstrap test
        log_res = bootstrap_test(log1, log2)
        cat_res = bootstrap_test(cat1, cat2)

        results.append({
            "Feature": feature,
            "Group 1": lvl1,
            "Group 2": lvl2,
            "n1": len(g1),
            "n2": len(g2),
            "Model": "Logistic",
            "Mean 1": log_res['mean_1'],
            "CI 1": log_res['ci_1'],
            "Mean 2": log_res['mean_2'],
            "CI 2": log_res['ci_2'],
            "p-value": log_res['p_value']
        })
        results.append({
            "Feature": feature,
            "Group 1": lvl1,
            "Group 2": lvl2,
            "n1": len(g1),
            "n2": len(g2),
            "Model": "CatBoost",
            "Mean 1": cat_res['mean_1'],
            "CI 1": cat_res['ci_1'],
            "Mean 2": cat_res['mean_2'],
            "CI 2": cat_res['ci_2'],
            "p-value": cat_res['p_value']
        })

    return pd.DataFrame(results)


Import the data, and written the feature level based on pervious infomation 

In [ ]:
import pandas as pd
df = pd.read_csv("datasets/rural I-40 Madison and Henderson crashes.csv")
# df = pd.read_csv("rural I-40 Madison and Henderson crashes.csv")

feature_levels = {
    "Presence of guardrails": [0, 1, 2, 3],
    "Cable barriers": [0, 1, 2, 3],
    "Rumble strips": [0, 1, 2, 3],
    "Pavement condition": [1, 2, 3, 4],  # exclude 0 = Unknown
    "Proximity to highway entrances and exits": [0, 1],
    "Urban vs. rural environment": [0, 1, 2],
    "Surrounding natural features": [0, 1, 2, 3, 4],
    "Shoulder type and width": [0, 1, 2, 3, 4],
    "Posted speed limit": [1, 2, 3, 4],
    "Presence and type of median or divider": [0, 1, 2, 3, 4],
    "Lane markings and signage visibility": [0, 1, 2, 3, 4],
    "Nighttime lighting and visibility": [0, 1, 2, 3]
}
geo_features = list(feature_levels.keys())  

In [24]:
all_results = []

for feature, levels in feature_levels.items():
    print(f"\nRunning test for: {feature}")
    try:
        result_df = run_multigroup_bootstrap_test(
            df,
            feature=feature,
            levels=levels,
            geo_features=geo_features
        )
        if result_df is not None:
            all_results.append(result_df)
    except Exception as e:
        print(f"Error in feature [{feature}]: {e}")



Running test for: Presence of guardrails

Running test for: Cable barriers

Running test for: Rumble strips

Running test for: Pavement condition

Running test for: Proximity to highway entrances and exits

Running test for: Urban vs. rural environment

Running test for: Surrounding natural features

Running test for: Shoulder type and width

Running test for: Posted speed limit

Running test for: Presence and type of median or divider

Running test for: Lane markings and signage visibility

Running test for: Nighttime lighting and visibility


In [25]:
final_results = pd.concat(all_results, ignore_index=True)

In [31]:
significant_results = final_results[
    (final_results["p-value"] < 0.05) &
    (final_results["n1"] > 0) &
    (final_results["n2"] > 0)
]
# print(significant_results)
significant_results.to_csv("significant_results.csv", index=False)


                                     Feature  Group 1  Group 2   n1   n2  \
0                     Presence of guardrails        0        1  553  198   
1                     Presence of guardrails        0        1  553  198   
2                     Presence of guardrails        0        2  553   48   
3                     Presence of guardrails        0        2  553   48   
4                     Presence of guardrails        1        2  198   48   
8                             Cable barriers        0        2  492   33   
9                             Cable barriers        0        2  492   33   
10                            Cable barriers        1        2  266   33   
11                            Cable barriers        1        2  266   33   
13                        Pavement condition        1        2  148   44   
14                        Pavement condition        1        3  148  557   
15                        Pavement condition        1        3  148  557   
16          

Presence of guardrails: shows significant difference in 1) None vs One side only; 2) None vs Both sides; 3) One side vs Both sides for Logistic only

Cable barriers: shows significant difference in 1) None vs Shoulder only; 2) Median only vs Shoulder only

Pavement condition: shows significant difference in 1) Poor vs Fair for CatBoost only; 2) Poor vs Good; 3) Poor vs Excellent; 4) Fair vs Good; 
                                                    5) Fair vs Excellent; 6) Good vs Excellent

Proximity to highway entrances and exits shows significant difference 

Surrounding natural features: shows significant difference in 1) Open/plain vs Forested/wooded; 2) Open/plain vs Mixed or complex terrain; 3) Forested/wooded vs Mixed or complex terrain for CatBoost only

Presence and type of median or divider: shows significant difference in 1) Grass median vs Raised concrete divider

Lane markings and signage visibility: shows significant difference in 1) Poor vs Clear and visible for Logistic; 2) Average vs Clear and visible

Nighttime lighting and visibility: shows significant difference in 1) No lighting vs Poor; 2) No lighting vs Moderate